In [ ]:
#working
from flask import Flask, Response
from flask_cors import CORS  # Import CORS from flask_cors
import cv2
import face_recognition
import os
import datetime

# Initialize Flask application
app = Flask(__name__)
CORS(app)  # Enable CORS for the Flask app

# Function to load known faces and their encodings
def load_known_faces(directory):
    known_faces = {}
    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            if face_encodings:
                # Use the filename without extension as the person's name
                name = os.path.splitext(filename)[0]
                known_faces[name] = face_encodings[0]
    return known_faces

# Directory containing known face images
known_faces_directory = "known_faces"
# Load known faces from directory
known_faces = load_known_faces(known_faces_directory)

# Directory to save detected face images
detected_faces_directory = "detected_faces"
os.makedirs(detected_faces_directory, exist_ok=True)

# Define a minimum buffer (in pixels) from the edges of the frame
BUFFER = 20

# Padding around the face bounding box (in pixels)
PADDING = 20

# Desired dimensions for the face image (400x400)
DESIRED_DIMENSIONS = (400, 400)

# Brightness adjustment factor
BRIGHTNESS_FACTOR = 1.2  # Increase brightness by 20%; adjust as needed

# Dictionary to keep track of presence status of known persons
is_present = {}

# Generator function to generate frames with detected faces
def generate_detected_frames():
    # Initialize video capture from the default webcam (0)
    video_capture = cv2.VideoCapture(0)
  
   # address="http://192.168.140.45:8080/video";
    #video_capture.open(address)
   # video_capture = cv2.VideoCapture()
    # Check if the webcam is opened successfully
    if not video_capture.isOpened():
        yield "Failed to open the webcam.", 500

    try:
        while True:
            # Capture frame-by-frame
            ret, frame = video_capture.read()

            # If there is a frame, proceed with face recognition
            if ret:
                # Convert the frame from BGR (OpenCV) to RGB (face_recognition)
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Find all face locations and encodings
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                # List to hold names for faces in this frame
                face_names = []

                # Track whether a face is recognized and its associated name
                for face_encoding in face_encodings:
                    # Initialize name as "Unknown"
                    name = "Unknown"

                    # Compare face encoding with known face encodings
                    for known_name, known_encoding in known_faces.items():
                        if face_recognition.compare_faces([known_encoding], face_encoding, tolerance=0.6)[0]:
                            name = known_name
                            break
                
                    face_names.append(name)

                # Process the results: save and display the frames
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Add padding to the bounding box coordinates
                    top = max(top - PADDING, 0)
                    right = min(right + PADDING, frame.shape[1])
                    bottom = min(bottom + PADDING, frame.shape[0])
                    left = max(left - PADDING, 0)
                    
                    # Check if the face is fully within the frame
                    if left > BUFFER and right < frame.shape[1] - BUFFER and top > BUFFER and bottom < frame.shape[0] - BUFFER:
                        # If the person was previously absent and is now present, update their image
                        if name != "Unknown":
                            # Check if the person was previously absent
                            if name not in is_present or not is_present[name]:
                                # Get the region of interest (ROI) of the face
                                roi_frame = frame[top:bottom, left:right]

                                # Resize the ROI frame for desired dimensions (400x400)
                                roi_frame_resized = cv2.resize(roi_frame, DESIRED_DIMENSIONS)

                                # Convert the resized image from BGR to HSV color space
                                hsv_frame = cv2.cvtColor(roi_frame_resized, cv2.COLOR_BGR2HSV)
                                
                                # Increase the brightness by multiplying the V channel
                                hsv_frame[:, :, 2] = cv2.multiply(hsv_frame[:, :, 2], BRIGHTNESS_FACTOR)
                                
                                # Convert the HSV frame back to BGR color space
                                roi_frame_brightened = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

                                # Use the person's name as the filename to replace old images with the same name
                                filename = f"{name}.jpg"

                                # Save the brightened and resized ROI frame (face image) to the detected_faces directory
                                cv2.imwrite(os.path.join(detected_faces_directory, filename), roi_frame_brightened)

                                # Get the current date and time
                                current_datetime = datetime.datetime.now()

                                # Construct response string with image name, date, and time
                                response_str = f"Image: {filename}, Date: {current_datetime.date().isoformat()}, Time: {current_datetime.time().isoformat()}"

                                # Yield the response string
                                yield response_str + '\n'

                                # Mark the person as present
                                is_present[name] = True

                    # Draw rectangle around the face for display purposes
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                    # Draw a label with the name below the face for display purposes
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

                # At the end of the frame processing, set all known persons as absent
                # Reset the presence status of names that were not seen in this frame
                for name in list(is_present.keys()):
                    if name not in face_names:
                        is_present[name] = False

                # Display the resulting image in a window
                cv2.imshow('Video', frame)

                # Break the loop on pressing 'q' key
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
           # else:
             #   print("Failed to capture frame.")
            #    break

    except Exception as e:
        yield f"An error occurred: {str(e)}", 500

    # Release the webcam and close all OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()

# Endpoint to start the webcam and perform face recognition
@app.route('/start-webcam', methods=['GET'])
def start_webcam():
    return Response(generate_detected_frames(), mimetype='text/plain')

# Home route
@app.route('/')
def home():
   return "Welcome to the Face Recognition API. Use the /start-webcam endpoint to start the webcam."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5011) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5011
 * Running on http://192.168.140.187:5011
Press CTRL+C to quit
127.0.0.1 - - [01/May/2024 16:49:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2024 16:49:25] "GET /start-webcam HTTP/1.1" 200 -
